# Fine-tune SantaCoder on YAML
Sources:
* https://github.com/loubnabnl/santacoder-finetuning
* https://colab.research.google.com/drive/1UMjeXHwOldpLnWjdm1499o2IYy0RgeTw?usp=sharing


### Install required libraries

In [ ]:
! pip install "transformers<4.41" datasets ipywidgets torch
! pip install accelerate -U
! pip install bitsandbytes -U

In [ ]:
import sys
import logging

nblog = open("1-train-yaml.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

In [ ]:
# In case very detailed information is needed from the training phase
# from transformers.utils import logging
# logging.set_verbosity_info()

### Log in to HuggingFace to access The Stack

In [ ]:
import os
from huggingface_hub import login
# Note: Prefer notebook_login() but this was not prompting properly
login(token=os.environ.get('HUGGINGFACE_TOKEN'))

### Import required libraries

In [ ]:
import torch
from dataclasses import dataclass
from datasets import load_dataset
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig
)
from dataset import ConstantLengthDataset
import pprint

In [ ]:
# BitsAndBytesConfig allows the configuration of the BitsAndBytes feature of Hugging Face Transformers.
# This feature enables efficient model inference by reducing the model size and computational requirements.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Enables loading the model in a 4-bit quantized format to reduce memory usage.
    bnb_4bit_use_double_quant=True,        # Activates double quantization, which quantizes not just the weights but also the activations.
    bnb_4bit_quant_type="nf4",             # Sets the quantization type to 'nf4', a 4-bit number format for quantization.
    bnb_4bit_compute_dtype=torch.bfloat16  # Specifies bfloat16 as the data type for computation, balancing precision and speed.
)

### Specify the pre-trained model and dataset

In [ ]:
model_id = "bigcode/santacoder"
dataset_id = "bigcode/the-stack-dedup"
data_dir = "data/yaml"

### Load the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

### Load the model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, use_cache=False)

### Load the dataset
Use the first 10% of the dataset

In [ ]:
dataset = load_dataset(dataset_id, data_dir=data_dir, split='train[:10%]')

### Calculate characters per token

In [ ]:
examples, total_characters, total_tokens = 500, 0, 0

for _, example in tqdm(zip(range(examples), iter(dataset)), total=examples):
    total_characters += len(example['content'])
    total_tokens += len(tokenizer(example['content']).tokens())

characters_per_token = total_characters / total_tokens
print(characters_per_token)

### Create a test split

In [ ]:
dataset = dataset.train_test_split(test_size=0.005, shuffle=False, seed=555)
train_ds = dataset["train"]
valid_ds = dataset["test"]
len(train_ds), len(valid_ds)

In [ ]:
train_ds = train_ds.shuffle(seed=555)

train_dataset = ConstantLengthDataset(
        tokenizer, train_ds, infinite=True, seq_length=1024, chars_per_token=characters_per_token 
    )
valid_dataset = ConstantLengthDataset(
        tokenizer, valid_ds, infinite=False, seq_length=1024, chars_per_token=characters_per_token
    )

### Define the training arguments

In [ ]:
training_args = TrainingArguments(
        output_dir="santacoder-finetuned-the-stack-yaml",
        dataloader_drop_last=True,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        optim="adafactor",
        evaluation_strategy="steps",
        # max_steps=5000,
        # max_steps=1000,
        # eval_steps=500,
        # save_steps=500,
        max_steps=100,
        eval_steps=50,
        save_steps=50,
        logging_steps=10,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        learning_rate=5e-5,
        lr_scheduler_type="cosine",
        warmup_steps=100,
        weight_decay=0.05,
        fp16=False,
        push_to_hub=False,
        logging_strategy='epoch',
        # load_best_model_at_end=True,
)

### Set up logging

In [ ]:
from transformers import EarlyStoppingCallback, TrainerCallback
import time
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

In [ ]:
current_timezone = 'Europe/Brussels'

class ETAProgressBarCallback(TrainerCallback):
    def __init__(self):
        self.start_time = None
        self.total_steps = None
        self.current_step = 0
        self.average_step_duration = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = datetime.now(ZoneInfo(current_timezone))
        self.total_steps = state.max_steps
        
        current_time = datetime.now(ZoneInfo(current_timezone))
        str_current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
        print(f"Starting training at {str_current_time}")

    def on_step_end(self, args, state, control, **kwargs):
        self.current_step += 1
        current_time = datetime.now(ZoneInfo(current_timezone))
        elapsed_time = current_time - self.start_time
        self.average_step_duration = elapsed_time / self.current_step
        estimated_total_time = self.average_step_duration * self.total_steps
        
        estimated_time_left = estimated_total_time - elapsed_time
        days = estimated_time_left.days
        hours, remainder = divmod(estimated_time_left.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        estimated_time_left_minutes_formatted = f"{days}d {hours:02d}h:{minutes:02d}m:{seconds:02d}s"
        
        estimated_end_time = (current_time + (estimated_total_time - elapsed_time)).strftime('%Y-%m-%d %H:%M:%S')

        # Calculate iterations per second
        it_per_sec = self.current_step / (elapsed_time.days*3600*24 + elapsed_time.seconds)
        # Access the current epoch from TrainerState
        current_epoch = state.epoch
        
        print(f"Step: {self.current_step}/{self.total_steps}, epoch: {current_epoch:.2f}, It/s: {it_per_sec:.2f}, est. time left: {estimated_time_left_minutes_formatted}, est. end time: {estimated_end_time}")
        
    def on_evaluate(self, args, state, control, model=None, metrics=None, **kwargs):
        # Log evaluation metrics after every evaluation phase
        if metrics:
            print("Evaluation metrics:")
            for key, value in metrics.items():
                print(f"{key}: {value}")

    def on_train_end(self, args, state, control, model=None, **kwargs):
        current_time = datetime.now(ZoneInfo(current_timezone))
        str_current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
        print("Training completed at {str_current_time}")

### Train

In [ ]:
train_dataset.start_iteration = 0

In [ ]:
trainer = Trainer(
    #tokenizer=tokenizer,
    model=model, args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=valid_dataset,
    callbacks=[
        # EarlyStoppingCallback(
        #     early_stopping_patience=5,
        #     early_stopping_threshold=1.0),
        ETAProgressBarCallback()]
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model() # this creates the config.json file etc you need to run generation tests

### Evaluate